In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import solve_ivp
from Constants import *
from Sterile_Nu_Parameters import *
from Momentum_Grid import *
from Tolerances import *

from DOF import *
from System_SterileDM_Prod import System_SterileDM_Prod
from Make_Rate_Table import Make_Table
from Asy_Redistribution import eqs_redist
from Thermo_quantities import Thermo_quantities
#Momentum grid for sterile neutrinos can be changed in line 7 of Momentum_Grid.py
#Sterile neutrino parameters and lepton asymmetries can be changed in Sterile_Nu_Parameters.py 

In [2]:
#Making table of the reaction rates

Gamma_a_Table = np.zeros((ni_Table,nT_Table)) #numbers of momentum and temperature grid
Gamma_a_anti_Table = np.zeros((ni_Table,nT_Table))

if (flavor == 'e'):

    La_ini = Le_ini
    
elif(flavor == 'mu'):
    
    La_ini = Lmu_ini
    
else:
    
    La_ini = Ltau_ini

Gamma_a_Table, Gamma_a_anti_Table = Make_Table(La_ini)

In [3]:
#Computing the sterile neutrino abundance (normalized by DM abundance)

#absolute tolerance and relative torelance
#These values needs to be tuned to solve in finite time due to numerical instability.

atol_ivp = 1e-13
rtol_ivp = 1e-4 


#Initial values

La_0 = np.zeros(1)

f_nus_0 = np.zeros(n)
f_nus_anti_0 = np.zeros(n)

T0 = 1e4 #MeV


if (flavor == 'e'):

    La_0[0] = Le_ini
    La_ini = Le_ini
    
elif(flavor == 'mu'):
    
    La_0[0] = Lmu_ini
    La_ini = Lmu_ini
    
    
else:
    
    La_0[0] = Ltau_ini
    La_ini = Ltau_ini

    

#Thermodynamic quantities at the beginning

x0 = np.array([0,0,0,0,0])

ans0 =  sp.optimize.root(eqs_redist, x0, args = (T0,La_ini), method='hybr',tol=tol_root)  

zeta_nue0 = ans0.x[0]
zeta_numu0 = ans0.x[1]
zeta_nutau0 = ans0.x[2]

zeta_e0 = ans0.x[0] - ans0.x[4]
zeta_mu0 = ans0.x[1] - ans0.x[4]
zeta_tau0 = ans0.x[2] - ans0.x[4]

zeta_B0 = ans0.x[3]

zeta_Q0 = ans0.x[4]


rho0, s0, P0, Delta_rho0, rho_nua0, rho_a0, Del_nua0, Del_a0, Del_nu_total0, Del_c_total0, Del_Q_QCD0 \
          = Thermo_quantities(T0,zeta_nue0,zeta_numu0,zeta_nutau0,zeta_B0,zeta_Q0)

s_ini = s0

sys_values_0 = np.concatenate((f_nus_0,f_nus_anti_0,La_0))


#Temperature range for numerical simulations

T_span = [1e4,15] 

#Temperature values to record the evolution
#To save memeories, we currently record the final values of the evolution.

#T_eval = np.logspace(1e4,15,1000)
T_eval = [15]

argList = [Gamma_a_Table,Gamma_a_anti_Table,s0,ms,Uas]


#Solving the system
#RK45 or LSODA might be better.

sol = solve_ivp(System_SterileDM_Prod,T_span,sys_values_0,t_eval=T_eval,args=argList,method='LSODA',atol=atol_ivp,rtol=rtol_ivp)


#Thermodynamic quantities at the end

l = len(sol.t)

x0 = np.array([0,0,0,0,0])

Tfin = 15
Lfin = sol.y[2*n,l-1]

ans =  sp.optimize.root(eqs_redist, x0, args = (Tfin,Lfin), method='hybr',tol=tol_root)  

zeta_nue = ans.x[0]
zeta_numu = ans.x[1]
zeta_nutau = ans.x[2]

zeta_e = ans.x[0] - ans.x[4]
zeta_mu = ans.x[1] - ans.x[4]
zeta_tau = ans.x[2] - ans.x[4]

zeta_B = ans.x[3]

zeta_Q = ans.x[4]

rho_fin, s_fin, P_fin, Delta_rho_fin, rho_nua_fin, rho_a_fin, Del_nua_fin, Del_a_fin, Del_nu_total_fin, Del_c_total_fin, Del_Q_QCD_fin \
          = Thermo_quantities(Tfin,zeta_nue,zeta_numu,zeta_nutau,zeta_B,zeta_Q)

#Computing the sterile neutrino abundance

l = len(sol.t)

n_nus = 0

for ni in range(n):

    y = y2[ni]
    dy = dy2[ni]

    n_nus += dy*y**2*(sol.y[ni,l-1] + sol.y[ni+n,l-1])

n_nus *= 1/(2*np.pi**2)*(s_fin/s_ini)

#Normalizing the abundance by the dark matter abundance today

T = sol.t[l-1]

gs = gs_below120MeV(T)

gs_0 = gs_below120MeV(1e-2)

s_0 = 2*np.pi**2/45*gs_0

T0 = 2.73*8.617*1e-5*1e-6 #K->eV->MeV #2.726?

h=0.673

rho_cr0 = 1.878*1e-29*h**2 #g cm^-3

rho_cr0 *= (0.511*3.862*1e-11)**3 # g cm^-3 -> g MeV^3
rho_cr0 *= 0.511/9.109*1e28 #g MeV^3->MeV^4

Omega_nus = ms*T0**3*(s_0/s_fin)/rho_cr0*n_nus*1/0.265

#Showing the sterile neutrino abundance
#1 is the correct DM abundance.

print(Omega_nus)

0.9336590417257681
